In [1]:
from PyQt5 import QtWidgets,uic
import sqlite3 as sql

In [2]:
def login():
    
    phone = homePage.usernameEnter.text()
    password = homePage.passwordEnter.text()
    
    con = sql.connect('demobank.db')
    
    query = """SELECT cust_name,cust_Account_num,cust_email,cust_balance
            FROM customer_details WHERE cust_phone = ? AND cust_password = ?"""
    
    ret = con.execute(query,(phone,password))
    
    ret = ret.fetchall()
    
    if len(ret)>0:
        homePage.close()
        custDashboard.welcome.setText('Welcome,'+ret[0][0])
        custDashboard.info.setText('''
        Account num : {}
        Balance : {}'''.format(ret[0][1],ret[0][3]))
        custDashboard.show()
    else:
        homePage.message.setText('invalid phone or password')
        homePage.usernameEnter.setText('')
        homePage.passwordEnter.setText('')
    

def gotosignup():
    homePage.close()
    signupPage.show()
    
def backfromsignup():
    signupPage.close()
    homePage.show()
    
def register():
    
    cust_name = signupPage.name.text()
    cust_password = signupPage.password.text()
    cust_email = signupPage.email.text()
    cust_phone = signupPage.phone.text()
    cust_dob = signupPage.dob.text()
    
    cust_account_num = cust_dob.split('-')[0]+cust_dob.split('-')[1]+cust_dob.split('-')[2]+cust_phone[-5:]
    
    cust_Id = cust_account_num[:5]+cust_name[:5]
    
    con = sql.connect('demobank.db')
    
    query = """INSERT INTO customer_details(cust_Id,cust_name,
    cust_Account_num, cust_email, cust_phone, cust_password,
    cust_dob,cust_balance) VALUES(?,?,?,?,?,?,?,?)"""
    
    con.execute(query,(cust_Id,cust_name,cust_account_num,cust_email,
                      cust_phone,cust_password,cust_dob,"2000.0"))
    con.commit()
    con.close()
    
    signupPage.name.setText(" ")
    signupPage.password.setText(" ")
    signupPage.email.setText(" ")
    signupPage.phone.setText(" ")
    signupPage.dob.setText(" ")
    signupPage.message.setText('registration successful')

def custlogout():
    custDashboard.close()
    homePage.usernameEnter.setText('')
    homePage.passwordEnter.setText('')
    homePage.show()

def opentransfer():
    transferDiag.show()
    

def transferamount():
    
    phonenum = transferDiag.phoneEnter.text()
    password = transferDiag.passwordEnter.text()
    payee_phone = transferDiag.payeePhone.text()
    amount = transferDiag.amount.text()
    con = sql.connect('demobank.db')
    
    query = """SELECT cust_Id, cust_Account_num ,cust_balance
            FROM customer_details WHERE cust_phone = ? AND cust_password = ?"""
    
    ret = con.execute(query,(phonenum,password))
    
    ret = ret.fetchall()
    
    if len(ret)>0:
        
        from_cust_Id = ret[0][0]
        from_cust_account = ret[0][1]
        from_cust_bal = ret[0][2]
        
    query = """SELECT cust_Id, cust_balance
            FROM customer_details WHERE cust_phone = ?"""
    
    ret = con.execute(query,(payee_phone,))
    ret = ret.fetchall()
    if len(ret)>0:
        to_cust_Id = ret[0][0]
        to_cust_bal = ret[0][1]
    
    if float(amount)<=float(from_cust_bal):
        new_from_cust_bal = str(float(from_cust_bal)-float(amount))
        new_to_cust_bal = str(float(to_cust_bal)+float(amount))
        query = """UPDATE customer_details SET cust_balance = ? WHERE cust_Id = ?"""
        con.execute(query,(new_to_cust_bal,to_cust_Id))
        
        query = """UPDATE customer_details SET cust_balance = ? WHERE cust_Id = ?"""
        con.execute(query,(new_from_cust_bal,from_cust_Id))
        
        cur = con.cursor()
        for i in cur.execute("""SELECT DATE('now')"""):
            dte = i[0]
        query = """INSERT INTO transaction_history(Cust_Id,date,type,amount) VALUES(?,?,?,?)"""
        con.execute(query,(to_cust_Id,dte,"deposit_by_transfer",amount))
        
        query = """INSERT INTO transaction_history(Cust_Id,date,type,amount) VALUES(?,?,?,?)"""
        con.execute(query,(from_cust_Id,dte,"withdrawl_by_transfer",amount))
        
        con.commit()
        con.close()
        
        custDashboard.info.setText('''
        Account num : {}
        Balance : {}'''.format(from_cust_account,str(new_from_cust_bal)))
        transferDiag.close()
        
    else:
        con.close()

def openwithdraw():
    
    withdrawDiag.show()
    
def withdrawamount():
    
    phonenum = withdrawDiag.phoneEnter.text()
    password = withdrawDiag.passwordEnter.text()
    amount = withdrawDiag.amount.text()
    
    con = sql.connect('demobank.db')
    
    query = """SELECT cust_Id, cust_Account_num ,cust_balance
            FROM customer_details WHERE cust_phone = ? AND cust_password = ?"""
    
    ret = con.execute(query,(phonenum,password))
    
    ret = ret.fetchall()
    
    if len(ret)>0:
        
        from_cust_Id = ret[0][0]
        from_cust_account = ret[0][1]
        from_cust_bal = ret[0][2]
    
    if float(from_cust_bal)>=float(amount):
        new_from_cust_bal = str(float(from_cust_bal)-float(amount))
        
        query = """UPDATE customer_details SET cust_balance = ? WHERE cust_Id = ?"""
        con.execute(query,(new_from_cust_bal,from_cust_Id))
        
        cur = con.cursor()
        for i in cur.execute("""SELECT DATE('now')"""):
            dte = i[0]
        query = """INSERT INTO transaction_history(Cust_Id,date,type,amount) VALUES(?,?,?,?)"""
        
        con.execute(query,(from_cust_Id,dte,"withdrawl",amount))
        con.commit()
        con.close()
        custDashboard.info.setText('''
        Account num : {}
        Balance : {}'''.format(from_cust_account,str(new_from_cust_bal)))
        withdrawDiag.close()
        
def view():
    
    cust_ac_num = custDashboard.info.text().strip().split(":")[1].split(" ")[1][:-1]
    
    con = sql.connect('demobank.db')
    
    query = """SELECT cust_Id FROM customer_details WHERE cust_Account_num = ?"""
    
    ret = con.execute(query,(cust_ac_num,))
    
    ret = ret.fetchall()
    
    if len(ret)>0:
        cust_Id = ret[0][0]
    
        query = """SELECT date,type,amount FROM transaction_history WHERE cust_Id=?"""

        ret = con.execute(query,(cust_Id,))
        
        df = dict()
        date_list = []
        type_list = []
        amount_list = []
        for i in ret:
            date_list.append([i[0]])
            type_list.append([i[1]])
            amount_list.append([i[2]])
        df.update({"date":date_list , "type": type_list , "amount":amount_list})
        
        ministatement.tableWidget.setRowCount(0) # initially no rows in table
    
        ministatement.tableWidget.setColumnCount(len(df.keys())) # add number of columns as per the df
        ministatement.tableWidget.setHorizontalHeaderLabels(df.keys()) # put their labels
        for i in range(len(list(df.keys()))):
        # insert the data as row items
            row_num = 0
            for data in df[list(df.keys())[i]]:
                if i == 0:
                    ministatement.tableWidget.insertRow(ministatement.tableWidget.rowCount())
                    row_num = ministatement.tableWidget.rowCount()-1
                ministatement.tableWidget.setItem(row_num , i, QtWidgets.QTableWidgetItem(str(data)))
                row_num += 1
                
        ministatement.show()


In [3]:
app = QtWidgets.QApplication([])

homePage = uic.loadUi('home.ui')
signupPage = uic.loadUi('signup.ui')
custDashboard = uic.loadUi('custDashboard.ui')
transferDiag = uic.loadUi("transfer.ui")
withdrawDiag = uic.loadUi('withdraw.ui')
ministatement = uic.loadUi('ministatement.ui')


homePage.show()

homePage.loginBut.clicked.connect(login)
homePage.signupBut.clicked.connect(gotosignup)

signupPage.back.clicked.connect(backfromsignup)
signupPage.signup.clicked.connect(register)


custDashboard.logout.clicked.connect(custlogout)
custDashboard.transfer.clicked.connect(opentransfer)
custDashboard.withdraw.clicked.connect(openwithdraw)
custDashboard.viewTransactions.clicked.connect(view)


transferDiag.confirmButton.clicked.connect(transferamount)

withdrawDiag.confirmButton.clicked.connect(withdrawamount)
app.exec()

0